In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'dgl'

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to C:\Users\Abdullah Al Amin\.dgl\cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [3]:
g = dataset[0]

print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [4]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


In [5]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [6]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.098 (best 0.098), test acc: 0.111 (best 0.111)
In epoch 5, loss: 1.886, val acc: 0.542 (best 0.552), test acc: 0.566 (best 0.574)
In epoch 10, loss: 1.800, val acc: 0.634 (best 0.634), test acc: 0.655 (best 0.655)
In epoch 15, loss: 1.688, val acc: 0.650 (best 0.650), test acc: 0.661 (best 0.658)
In epoch 20, loss: 1.553, val acc: 0.676 (best 0.676), test acc: 0.680 (best 0.680)
In epoch 25, loss: 1.397, val acc: 0.714 (best 0.714), test acc: 0.715 (best 0.715)
In epoch 30, loss: 1.227, val acc: 0.728 (best 0.728), test acc: 0.736 (best 0.736)
In epoch 35, loss: 1.052, val acc: 0.742 (best 0.742), test acc: 0.745 (best 0.745)
In epoch 40, loss: 0.881, val acc: 0.758 (best 0.758), test acc: 0.752 (best 0.752)
In epoch 45, loss: 0.723, val acc: 0.758 (best 0.760), test acc: 0.762 (best 0.757)
In epoch 50, loss: 0.586, val acc: 0.772 (best 0.772), test acc: 0.765 (best 0.765)
In epoch 55, loss: 0.472, val acc: 0.770 (best 0.772), test acc: 0.769 (best 0